## PROBLEM

## Problem statement

Prepare 4 random 4-qubit quantum states of your choice.

Create and train a variational circuit that transforms input states into predefined output states. Namely

if random state 1 is provided, it returns state |0011>

if random state 2 is provided, it returns state |0101>

if random state 3 is provided, it returns state |1010>

if random state 4 is provided, it returns state |1100>

What would happen if you provided a different state?

Analyze and discuss the results.

## Solution

Let's divide the big and difficult problem into tiny subproblems that are easier to solve. First, the problem is asking us to prepare a 4 random 4 qubit quantum sistem. We can easily do that by just calling qiskit random circuit and specifying that we want 4 qubits. We can see this in the following code:

In [3]:
#Necessary imports
from qiskit import QuantumCircuit, execute
from qiskit.circuit.random import random_circuit
import random
from qiskit import Aer
from qiskit.aqua.components.optimizers import ADAM
import numpy as np

#Generate 4 random circuits of 4 qubits.
#The function receives 4 seeds to get the same random circuits given
#the same seed.
#The function returns a list containing 4 random circuits.
def generate_random_circuits(seed1,seed2,seed3,seed4):
    qc1 = random_circuit(4, 2, max_operands=3, seed=seed1)
    qc2 = random_circuit(4, 2, max_operands=3, seed=seed2)
    qc3 = random_circuit(4, 2, max_operands=3, seed=seed3)
    qc4 = random_circuit(4, 2, max_operands=3, seed=seed4)
    return [qc1,qc2,qc3,qc4]

/home/luis/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Now the problem is asking us several things, the first is create a variation circuit that transform predefined input states into predefined output states. Let's have a closer look at this.
We need a variational circuit that is capable of transforming a input state into another. This first part looks difficult until we saw the fact that states in quantum mechanics are really similar to probability distributions, so we can think this problem as: given a probability distribution we want to train a quantum circuit that outputs numbers from that probability distribution. Luckily for us, this problem had been aproach in the past by researchers [1] [2]. So we can take their anzats and apply it to our problem. The following function does exactly that, it constructs an anzats based of the proposed anzats from [1]:

In [4]:
#Construct the anzats
#The function receives the number of qubits used in the anzats, the number of
#layers that we want in our anzats and the parameters that
#are used.
#The function returns the anzats.
def generate_anzats(n_qubits,n_layers,parameters):
    circuit = QuantumCircuit(n_qubits)

    for layer in range(n_layers):
        for i in range(n_qubits):
            circuit.ry(parameters[n_qubits*layer+i], i)

        for i in range(n_qubits):
            circuit.cz(i,(i+1)%n_qubits)

    for i in range(n_qubits):
        circuit.ry(parameters[n_qubits*(n_layers)+i], i)
    circuit.barrier()
    return circuit

Now that we have our circuit, we have to train it. To train any machine learning algorithm (yes, even the quantum ones) we need three basic ingredients that are:
- A dataset and their objective values (only applies for supervised learning) 
- An objective function that we want to minimice/maximaze
- An optimizer that takes the objective function and it returns the optimal parameters.

Let's grab these points in order, firts we need to pick the dataset. In this case, the dataset is the random input states that we did earlier and the labels or the objective values are the expected output values that were mentioned in the problem formulation. Next we need an objective function. Luckily for us (once again) researchers have proposed a cost function that works well to measure a model accuaracy between two probability distributions. This is the negative log function [2], However something is missing... We want to compare two probability distributions but we only have quantum states, well we can apply the same argument here to "treat" these quantum states as probability distributions. Nonetheless we need to know the statevector of our random circuit in order to propose it as a probability distribution. So in order to approximate our state vector, we need to do a little bit of sampling. We can see these two things (sampling and the cost function) in the following cell:

In [5]:
# Function to calculate the cost function. This cost function
# uses the negative log cost function.
def objective_function(circuit, label, params, epsilon=0.0005):
    cost_iteration = 0
    #for _ in range(0,100):
    new_circuit=circuit.copy()
    new_circuit.append(generate_anzats(n,num_layers,params),[0,1,2,3])
    distr = get_prob_distribution(new_circuit, Aer.get_backend('qasm_simulator'))
    if(label in distr):
        cost_iteration += np.log(max(epsilon,distr[label]))
    else:
        cost_iteration += np.log(epsilon)            
    print("the cost of this iteration was: " + str(-cost_iteration) + " and the params were: " + str(params))
    return -cost_iteration

# Function that does the sampling necessary to get an approximation
# of a state vector.
def get_prob_distribution(bound_circuit,backend):
    #Assign parameters using the assign_parameters method
    #print(len(params), circuit.num_parameters)
    bound_circuit.measure_all()
    raw_distrubution = execute(bound_circuit,backend,shots=1000).result().get_counts()
    for key in raw_distrubution.keys():
        raw_distrubution[key] = raw_distrubution[key]/1000
    return raw_distrubution

The last thing we need to do is to choose the optimizer to optimize the parameters with respecte to the cost function. We choose an ADAM (amsgrad) optimized with a really low noise factor taking into account what is proposed is [1]. However, we will change the learning rate to 0.1. This was choose empirically because we saw a really slow converge using the leraning rate that is proposed in [1].

In [6]:
def train(circuit, label, init_params, iterations, learning_rate):
    optimizer = ADAM(maxiter=iterations, lr=learning_rate, amsgrad=True, noise_factor=1e-020)
    object_funct = lambda params: objective_function(circuit, label, params)
    opt_params, value, _ = optimizer.optimize(len(init_params), object_funct, initial_point=init_params)
    return opt_params, value

Excellent, we have all the ingredients that we need. Now, it's time to do some experiments in the next section.

## Random state one

To show the above process in action let's train our first random circuit to output the state |0011>. But first, let's choose a random set of parameters to start our training. Given the fact that we don't know which is a good starting point we can just choose a random set of parameters. So that's what we are going to do in the next cells, first we are going to choose a random set of paramters that are random numbers in a range of 0 to 20 and then we are going to train our circuit for 200 iterations.

In [7]:
#Creation of some parameters necesary for all the executions
seed1 = 1
seed2 = 2
seed3 = 3
seed4 = 4
n = 4
num_layers = 3
circuits = generate_random_circuits(seed1,seed2,seed3,seed4)
labels = ["0011","0101", "1010", "1100"]
param_list = [random.randint(0,20) for _ in range(0,(n*(num_layers+1)))]
print("the initial parameters were: " + str(param_list))

the initial parameters were: [8, 20, 19, 15, 20, 0, 9, 0, 17, 17, 8, 20, 10, 17, 16, 15]


In [15]:
#200 iterations and 0.1
first = circuits[0]
opt_parameters,value = train(first, "0011", param_list, 300, 0.1)

the cost of this iteration was: 3.575550768806933 and the params were: [7, 17, 5, 1, 3, 7, 17, 20, 10, 5, 3, 4, 12, 16, 8, 5]
the cost of this iteration was: 2.8824035882469876 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.146555163288575 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.036554268074246 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.2441936328524905 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.079113882493042 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.2188758248682006 and the params were: [ 7. 17.  5.  1.  3.  7. 17. 20. 10.  5.  3.  4. 12. 16.  8.  5.]
the cost of this iteration was: 3.101092789211817

And now let's have a look to see the probability distribution that our circuit is outputing.

In [16]:
bound_circuit = first.copy()
bound_circuit.append(generate_anzats(n,num_layers,opt_parameters),[0,1,2,3])
distr = get_prob_distribution(bound_circuit,Aer.get_backend('qasm_simulator'))
print("The probability distribution is: " +str(distr))

The probability distribution is: {'1110': 0.006, '1011': 0.019, '0110': 0.021, '0100': 0.013, '0010': 0.069, '1001': 0.069, '0111': 0.004, '0000': 0.02, '0011': 0.154, '1101': 0.069, '0001': 0.095, '1100': 0.021, '0101': 0.087, '1111': 0.238, '1010': 0.115}


And We can see that while we didn't get a perfect probability distribution that outputs the state 0011 everytime, we get the probability to measure that state with a really high probability.

## SOLUTION

Now that we whow how to train a circuit, we can create the function that solves the problem statement. That is, given a state we want to get the variational circuit that outpouts the correct result. The function is as follows.

In [12]:
def task_2(state_number):
    params = [random.randint(0,20) for _ in range(0,(n*(num_layers+1)))]
    if(state_number == 1):
        bound_circuit = circuits[0].copy()
        opt_param,value = train(circuits[0], "0011", params, iterations=10, learning_rate=0.2)
        label = labels[0]
    elif(state_number == 2):
        bound_circuit = circuits[1].copy()
        opt_param,value = train(circuits[1], "0101", params, iterations=10, learning_rate=0.2)
        label = labels[1]
    elif(state_number == 3):
        bound_circuit = circuits[2].copy()
        opt_param,value = train(circuits[2], "1010", params, iterations=10, learning_rate=0.2)
        label = labels[2]
    else:
        bound_circuit = circuits[3].copy()
        opt_param,value = train(circuits[3], "1100", params, iterations=10, learning_rate=0.2)
        label = labels[3]
    bound_circuit.append(generate_anzats(n,num_layers,opt_param),[0,1,2,3])
    distr = get_prob_distribution(bound_circuit,Aer.get_backend('qasm_simulator'))
    print("\n \n \n The probability distribution to output the " + str(label) + " state is: " +str(distr))
    return opt_param,value

Now let's try it. 

In [14]:
_ = task_2(2)

the cost of this iteration was: 1.8451602459551701 and the params were: [18, 5, 6, 19, 19, 7, 19, 12, 9, 2, 16, 12, 2, 9, 8, 9]
the cost of this iteration was: 1.851509473633829 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 1.9449106487222299 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 1.8388510767619055 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 2.0874737133771 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 1.8838747581358606 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 1.7897614665653818 and the params were: [18.  5.  6. 19. 19.  7. 19. 12.  9.  2. 16. 12.  2.  9.  8.  9.]
the cost of this iteration was: 1.98777435315401

## DISCUSSION
Now that we saw the results let's review some interesting things that we like to point out. These things range from optimization in the methods to a littli bit of discussion for the proposed methods.

### OPTIMIZATIONS
If you saw carefully the cost function we didn't actually do a dot product like you would expect with these problems. This was for the following reasons. First, we know that the only entry in the probability distribution for the objective states is always one. Second, we know that if we do the dot product between this vector and any other vector we are going to get the state entry of the second vector. So instead of doing this, and wasting a linear time algorithm, we can actually make this product in constant time by just accessing a dictionary in the state that we are interested in.

### What would happen if you provided a different state.
My implementation is robust ebough in a sense that even if you provided me a different objective state, we could train the variational circuit without changing any of the code presented above. However, we could only do this if you provide a state that is not in any superposition of states and by calling the train method directly (otherwise you'll get the state four). This is due to the fact that we introduce an optimization to do the cost function. So let's put an example that our model generalice well enough. Let's suppose that you have a fifth random state that you want to transform into the output state |1111>. We can do this easily by just:

In [11]:
opt_parameters,value = train(random_circuit(4, 2, max_operands=3), "1111", param_list, 100, 0.1)

the cost of this iteration was: 3.473768074496991 and the params were: [8, 20, 19, 15, 20, 0, 9, 0, 17, 17, 8, 20, 10, 17, 16, 15]
the cost of this iteration was: 3.036554268074246 and the params were: [ 8. 20. 19. 15. 20.  0.  9.  0. 17. 17.  8. 20. 10. 17. 16. 15.]
the cost of this iteration was: 3.3813947543659757 and the params were: [ 8. 20. 19. 15. 20.  0.  9.  0. 17. 17.  8. 20. 10. 17. 16. 15.]
the cost of this iteration was: 3.123565645063876 and the params were: [ 8. 20. 19. 15. 20.  0.  9.  0. 17. 17.  8. 20. 10. 17. 16. 15.]
the cost of this iteration was: 3.101092789211817 and the params were: [ 8. 20. 19. 15. 20.  0.  9.  0. 17. 17.  8. 20. 10. 17. 16. 15.]
the cost of this iteration was: 3.2968373663379125 and the params were: [ 8. 20. 19. 15. 20.  0.  9.  0. 17. 17.  8. 20. 10. 17. 16. 15.]
the cost of this iteration was: 2.8824035882469876 and the params were: [8.0e+00 2.0e+01 1.9e+01 1.5e+01 2.0e+01 1.0e-10 9.0e+00 0.0e+00 1.7e+01
 1.7e+01 8.0e+00 2.0e+01 1.0e+01 1.7e

## Initial parameters matter
We can see that the same variational circuit with the same parameters for the optimizer gave us way different results. This is partially due to the initial parameters that we gave the model in the first iteration. So if we knew in advace what are a good set of initial parameters we could get better results.

## REFERENCES:
- [1] Zoufal, C., Lucchi, A. & Woerner, S. Quantum Generative Adversarial Networks for learning and loading random distributions. npj Quantum Inf 5, 103 (2019). https://doi.org/10.1038/s41534-019-0223-2
- [2] Benedetti, M., Garcia-Pintos, D., Perdomo, O. et al. A generative modeling approach for benchmarking and training shallow quantum circuits. npj Quantum Inf 5, 45 (2019). https://doi.org/10.1038/s41534-019-0157-8